# 「8」の画像識別を可視化する

手書き数字「8」がCNNでどのように処理されるか、各ステップを可視化します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

# 日本語フォント設定（Colabの場合）
plt.rcParams['font.size'] = 10

## Step 1: 「8」の画像を作成

In [ ]:
# 24×24の「8」を手動で作成
def create_digit_8():
    img = np.zeros((24, 24))
    
    # 上の丸
    for i in range(4, 10):
        for j in range(8, 16):
            # 楕円形の上部
            if ((i - 6.5) ** 2) / 4 + ((j - 12) ** 2) / 12 < 1.5:
                img[i, j] = 200
            # 内側を黒く
            if ((i - 6.5) ** 2) / 2 + ((j - 12) ** 2) / 6 < 1:
                img[i, j] = 0
    
    # 下の丸
    for i in range(10, 18):
        for j in range(7, 17):
            # 楕円形の下部
            if ((i - 14) ** 2) / 10 + ((j - 12) ** 2) / 16 < 1.5:
                img[i, j] = 200
            # 内側を黒く
            if ((i - 14) ** 2) / 5 + ((j - 12) ** 2) / 8 < 1:
                img[i, j] = 0
    
    # より8らしくするため手動調整
    img[4:6, 10:14] = 220  # 上部
    img[8:10, 10:14] = 220  # 中央上
    img[10:12, 9:15] = 220  # 中央
    img[16:18, 9:15] = 220  # 下部
    img[5:9, 8:10] = 220   # 左上
    img[5:9, 14:16] = 220  # 右上
    img[11:17, 7:9] = 220  # 左下
    img[11:17, 15:17] = 220 # 右下
    
    return img

# もう少しきれいな「8」を作成
def create_clean_digit_8():
    img = np.zeros((24, 24))
    
    # 上の丸（外側）
    cy1, cx = 7, 12
    for i in range(24):
        for j in range(24):
            dist = np.sqrt((i - cy1)**2 + (j - cx)**2)
            if 2.5 < dist < 4.5:
                img[i, j] = max(img[i, j], 220)
    
    # 下の丸（外側）
    cy2 = 15
    for i in range(24):
        for j in range(24):
            dist = np.sqrt((i - cy2)**2 + (j - cx)**2)
            if 3 < dist < 5:
                img[i, j] = max(img[i, j], 220)
    
    return img

digit_8 = create_clean_digit_8()

plt.figure(figsize=(6, 6))
plt.imshow(digit_8, cmap='gray', vmin=0, vmax=255)
plt.title('Step 1: Input Image "8" (24x24)', fontsize=14, fontweight='bold')
plt.colorbar(label='Pixel Value (0=black, 255=white)')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

print(f"Image shape: {digit_8.shape}")
print(f"Pixel range: {digit_8.min():.0f} - {digit_8.max():.0f}")

## Step 2: 畳み込み（フィルタによる特徴抽出）

エッジ検出フィルタを適用して、「8」の輪郭を検出します。

In [ ]:
# エッジ検出フィルタを定義

# フィルタ1: 縦エッジ検出（Sobel vertical）
filter_vertical = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
]) / 4

# フィルタ2: 横エッジ検出（Sobel horizontal）
filter_horizontal = np.array([
    [-1, -2, -1],
    [ 0,  0,  0],
    [ 1,  2,  1]
]) / 4

# フィルタを可視化
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

ax = axes[0]
im = ax.imshow(filter_vertical, cmap='RdBu', vmin=-1, vmax=1)
ax.set_title('Filter 1: Vertical Edge\n(縦エッジ検出)', fontsize=11, fontweight='bold')
for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{filter_vertical[i,j]:.1f}', ha='center', va='center', fontsize=12)
ax.set_xticks([])
ax.set_yticks([])

ax = axes[1]
im = ax.imshow(filter_horizontal, cmap='RdBu', vmin=-1, vmax=1)
ax.set_title('Filter 2: Horizontal Edge\n(横エッジ検出)', fontsize=11, fontweight='bold')
for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{filter_horizontal[i,j]:.1f}', ha='center', va='center', fontsize=12)
ax.set_xticks([])
ax.set_yticks([])

plt.suptitle('Step 2: Convolution Filters', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 畳み込み演算を実行
from scipy.ndimage import convolve

# 畳み込み適用
feature_map_v = convolve(digit_8, filter_vertical)
feature_map_h = convolve(digit_8, filter_horizontal)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 入力画像
ax = axes[0]
ax.imshow(digit_8, cmap='gray', vmin=0, vmax=255)
ax.set_title('Input: "8"\n(24x24)', fontsize=12, fontweight='bold')
ax.set_xlabel('Original Image')

# 縦エッジ特徴マップ
ax = axes[1]
im = ax.imshow(feature_map_v, cmap='RdBu')
ax.set_title('Feature Map 1\nVertical Edges (縦エッジ)', fontsize=12, fontweight='bold')
ax.set_xlabel('Left=Blue(-), Right=Red(+)')
plt.colorbar(im, ax=ax, shrink=0.8)

# 横エッジ特徴マップ
ax = axes[2]
im = ax.imshow(feature_map_h, cmap='RdBu')
ax.set_title('Feature Map 2\nHorizontal Edges (横エッジ)', fontsize=12, fontweight='bold')
ax.set_xlabel('Top=Blue(-), Bottom=Red(+)')
plt.colorbar(im, ax=ax, shrink=0.8)

plt.suptitle('Step 2: Convolution Results (畳み込み結果)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("縦エッジ: 8の左右の縦線を検出（青=左エッジ、赤=右エッジ）")
print("横エッジ: 8の上中下の横線を検出（青=上エッジ、赤=下エッジ）")

## Step 3: ReLU + Max Pooling

In [ ]:
# ReLU関数
def relu(x):
    return np.maximum(0, x)

# Max Pooling (2x2)
def max_pool_2x2(x):
    h, w = x.shape
    new_h, new_w = h // 2, w // 2
    output = np.zeros((new_h, new_w))
    for i in range(new_h):
        for j in range(new_w):
            output[i, j] = np.max(x[i*2:i*2+2, j*2:j*2+2])
    return output

# ReLU適用
relu_v = relu(feature_map_v)
relu_h = relu(feature_map_h)

# Pooling適用
pooled_v = max_pool_2x2(relu_v)
pooled_h = max_pool_2x2(relu_h)

In [ ]:
# ReLU + Poolingの可視化
fig, axes = plt.subplots(2, 3, figsize=(14, 9))

# 上段: 縦エッジ
ax = axes[0, 0]
im = ax.imshow(feature_map_v, cmap='RdBu')
ax.set_title('Conv Output\n(縦エッジ)', fontsize=11)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[0, 1]
im = ax.imshow(relu_v, cmap='Reds')
ax.set_title('After ReLU\n(負の値を0に)', fontsize=11)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[0, 2]
im = ax.imshow(pooled_v, cmap='Reds')
ax.set_title(f'After Max Pool\n({pooled_v.shape[0]}x{pooled_v.shape[1]})', fontsize=11)
plt.colorbar(im, ax=ax, shrink=0.7)

# 下段: 横エッジ
ax = axes[1, 0]
im = ax.imshow(feature_map_h, cmap='RdBu')
ax.set_title('Conv Output\n(横エッジ)', fontsize=11)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[1, 1]
im = ax.imshow(relu_h, cmap='Reds')
ax.set_title('After ReLU\n(負の値を0に)', fontsize=11)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[1, 2]
im = ax.imshow(pooled_h, cmap='Reds')
ax.set_title(f'After Max Pool\n({pooled_h.shape[0]}x{pooled_h.shape[1]})', fontsize=11)
plt.colorbar(im, ax=ax, shrink=0.7)

# 矢印を追加
for row in range(2):
    for col in range(2):
        axes[row, col].annotate('', xy=(1.15, 0.5), xytext=(1.02, 0.5),
                                 xycoords='axes fraction', textcoords='axes fraction',
                                 arrowprops=dict(arrowstyle='->', color='green', lw=2))

axes[0, 0].set_ylabel('Filter 1\n(縦エッジ)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Filter 2\n(横エッジ)', fontsize=12, fontweight='bold')

plt.suptitle('Step 3: ReLU + Max Pooling (24x24 → 12x12)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"サイズ変化: {feature_map_v.shape} → {relu_v.shape} → {pooled_v.shape}")

## Step 4: 2回目の畳み込み（より複雑な特徴）

1回目で検出したエッジを組み合わせて、「丸い形」などを検出します。

In [ ]:
# 2回目の畳み込み用フィルタ
# 「丸い形」を検出するフィルタ（簡略化）

# フィルタ3: 左上から右下への曲線
filter_curve1 = np.array([
    [2, 1, 0],
    [1, 2, 1],
    [0, 1, 2]
]) / 6

# フィルタ4: 右上から左下への曲線
filter_curve2 = np.array([
    [0, 1, 2],
    [1, 2, 1],
    [2, 1, 0]
]) / 6

# 2つの特徴マップを積み重ね
stacked_features = np.stack([pooled_v, pooled_h], axis=-1)

# 簡略化: 2つの特徴マップの合成に対して畳み込み
combined_feature = pooled_v + pooled_h

# 2回目の畳み込み
conv2_out1 = convolve(combined_feature, filter_curve1)
conv2_out2 = convolve(combined_feature, filter_curve2)

# ReLU
relu2_1 = relu(conv2_out1)
relu2_2 = relu(conv2_out2)

# Pooling
pooled2_1 = max_pool_2x2(relu2_1)
pooled2_2 = max_pool_2x2(relu2_2)

In [ ]:
# Step 4の可視化
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# 入力（1回目のPooling結果を合成）
ax = axes[0, 0]
im = ax.imshow(combined_feature, cmap='Reds')
ax.set_title('Input to Conv2\n(Combined Features)', fontsize=10)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[1, 0]
ax.axis('off')
ax.text(0.5, 0.5, 'Vertical +\nHorizontal\nEdges', ha='center', va='center', 
        fontsize=11, transform=ax.transAxes)

# 2回目の畳み込み結果
ax = axes[0, 1]
im = ax.imshow(conv2_out1, cmap='RdBu')
ax.set_title('Conv2 Filter1\n(Curve Pattern)', fontsize=10)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[1, 1]
im = ax.imshow(conv2_out2, cmap='RdBu')
ax.set_title('Conv2 Filter2\n(Opposite Curve)', fontsize=10)
plt.colorbar(im, ax=ax, shrink=0.7)

# ReLU後
ax = axes[0, 2]
im = ax.imshow(relu2_1, cmap='Reds')
ax.set_title('After ReLU', fontsize=10)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[1, 2]
im = ax.imshow(relu2_2, cmap='Reds')
ax.set_title('After ReLU', fontsize=10)
plt.colorbar(im, ax=ax, shrink=0.7)

# Pooling後
ax = axes[0, 3]
im = ax.imshow(pooled2_1, cmap='Reds')
ax.set_title(f'After Pool\n({pooled2_1.shape[0]}x{pooled2_1.shape[1]})', fontsize=10)
plt.colorbar(im, ax=ax, shrink=0.7)

ax = axes[1, 3]
im = ax.imshow(pooled2_2, cmap='Reds')
ax.set_title(f'After Pool\n({pooled2_2.shape[0]}x{pooled2_2.shape[1]})', fontsize=10)
plt.colorbar(im, ax=ax, shrink=0.7)

plt.suptitle('Step 4: Second Convolution (より複雑な特徴の検出)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"2回目のPooling後サイズ: {pooled2_1.shape}")

## 全体の流れを1枚で可視化

In [ ]:
# 全体フローの可視化
fig, axes = plt.subplots(2, 6, figsize=(18, 7))

# 上段: 処理の流れ
stages = [
    (digit_8, 'Input\n"8"\n24x24', 'gray'),
    (feature_map_v, 'Conv1\nVertical', 'RdBu'),
    (relu_v, 'ReLU', 'Reds'),
    (pooled_v, 'Pool1\n12x12', 'Reds'),
    (relu2_1, 'Conv2+ReLU', 'Reds'),
    (pooled2_1, 'Pool2\n6x6', 'Reds'),
]

for idx, (data, title, cmap) in enumerate(stages):
    ax = axes[0, idx]
    im = ax.imshow(data, cmap=cmap)
    ax.set_title(title, fontsize=10, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    if idx < len(stages) - 1:
        ax.annotate('', xy=(1.2, 0.5), xytext=(1.05, 0.5),
                    xycoords='axes fraction', textcoords='axes fraction',
                    arrowprops=dict(arrowstyle='->', color='green', lw=2))

# 下段: 横エッジの流れ
stages2 = [
    (digit_8, '', 'gray'),
    (feature_map_h, 'Conv1\nHorizontal', 'RdBu'),
    (relu_h, 'ReLU', 'Reds'),
    (pooled_h, 'Pool1\n12x12', 'Reds'),
    (relu2_2, 'Conv2+ReLU', 'Reds'),
    (pooled2_2, 'Pool2\n6x6', 'Reds'),
]

for idx, (data, title, cmap) in enumerate(stages2):
    ax = axes[1, idx]
    if idx == 0:
        ax.axis('off')
        continue
    im = ax.imshow(data, cmap=cmap)
    ax.set_title(title, fontsize=10, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    if idx < len(stages2) - 1:
        ax.annotate('', xy=(1.2, 0.5), xytext=(1.05, 0.5),
                    xycoords='axes fraction', textcoords='axes fraction',
                    arrowprops=dict(arrowstyle='->', color='green', lw=2))

axes[0, 0].set_ylabel('Filter 1\n(縦)', fontsize=11)
axes[1, 1].set_ylabel('Filter 2\n(横)', fontsize=11)

plt.suptitle('CNN Processing Flow: "8" → Feature Extraction', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 「8」の特徴がどこで検出されているか

In [ ]:
# 「8」の特徴検出を可視化
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# 元画像
ax = axes[0]
ax.imshow(digit_8, cmap='gray')
ax.set_title('Original "8"', fontsize=12, fontweight='bold')
ax.set_xticks([])
ax.set_yticks([])

# 縦エッジ（左右の線を検出）
ax = axes[1]
ax.imshow(digit_8, cmap='gray', alpha=0.3)
ax.imshow(np.abs(feature_map_v), cmap='Reds', alpha=0.7)
ax.set_title('Vertical Edges\n(8の左右の線)', fontsize=12, fontweight='bold')
ax.set_xticks([])
ax.set_yticks([])

# 横エッジ（上中下の線を検出）
ax = axes[2]
ax.imshow(digit_8, cmap='gray', alpha=0.3)
ax.imshow(np.abs(feature_map_h), cmap='Blues', alpha=0.7)
ax.set_title('Horizontal Edges\n(8の上中下の線)', fontsize=12, fontweight='bold')
ax.set_xticks([])
ax.set_yticks([])

# 合成（8の全体構造）
ax = axes[3]
combined = np.abs(feature_map_v) + np.abs(feature_map_h)
ax.imshow(digit_8, cmap='gray', alpha=0.3)
ax.imshow(combined, cmap='hot', alpha=0.7)
ax.set_title('Combined\n(8の輪郭全体)', fontsize=12, fontweight='bold')
ax.set_xticks([])
ax.set_yticks([])

plt.suptitle('Where CNN Detects Features of "8"', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("赤: 縦エッジ（8の左右の縦線）")
print("青: 横エッジ（8の上部・中央・下部の横線）")
print("黄: 両方が強く反応している部分（8の特徴的な形状）")

## まとめ: 「8」識別の流れ

```
Step 1: 入力 (24×24)
    ↓
Step 2: 畳み込み1 (フィルタで縦・横エッジ検出)
    ↓  
Step 3: ReLU + Pooling (負値カット + サイズ縮小)
    ↓
Step 4: 畳み込み2 (エッジを組み合わせて「丸」を検出)
    ↓
Step 5: ReLU + Pooling
    ↓
Step 6: 平坦化 + 全結合層
    ↓
Step 7: Softmax → P(8) = 91%
```

**ポイント**: 「8」は上下に丸があり中央でくびれている。CNNはこの特徴的なパターンを段階的に検出している！